<a href="https://colab.research.google.com/github/nakib103/pancancer_classification/blob/master/conv_2D_with_modularity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import packages

import numpy as np
import pandas as pd
import pickle
import math

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras import regularizers
from tensorflow.keras import optimizers
from tensorflow.keras.utils import to_categorical

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [ ]:
# run this cell to mount your Google Drive.

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# load data and label

data = pd.read_pickle("/content/drive/My Drive/data/data_df.pkl")
label = pd.read_pickle("/content/drive/My Drive/data/label_df.pkl")

In [ ]:
def preprocess_data(data):
  # create data variable from loaded data
  data_log_transformed = data.apply(lambda x: np.log2(x + 1))

  return data_log_transformed

In [ ]:
def shuffle_data(data, label):
  ## same patient check
  # generate shuffled index
  shuffled_index = np.random.permutation(data.index)

  # shiffle data and label
  data_shuffled = data.reindex(shuffled_index)
  label_shuffled = label.reindex(shuffled_index)

  return data_shuffled, label_shuffled

In [ ]:
def data_pd_to_np(data, shape):
  # convert dataframe to numpy array
  x = data.iloc[:].values

  # pad the training postion with zero
  x = np.pad(x, ((0, 0),(0, shape[0] * shape[1] - x.shape[1])), 'constant', constant_values=0)

  # reshape to 2D
  x = np.reshape(x, (x.shape[0], 1, shape[0], shape[1]))

  # normalize data to [0, 255] range
  x = ( x / np.max(x) ) * 255

  # convert data format to integer
  x = x.astype(int)
  
  return x

In [ ]:
def label_pd_to_np(label):
  # create label variable from loaded label
  y = label.iloc[:].values

  for id_y, lab_y in enumerate(y):
    for id_class, lab_class in enumerate(classes):
      if lab_y[0] == lab_class:
        y[id_y] = id_class
        break
  print(y.shape)
  y = y[:,0]
  y = to_categorical(y)

  return y

In [ ]:
# store the 2D data and corresponding label

# np.save('/content/drive/My Drive/data/data_np_2D.npy', x)
# np.save('/content/drive/My Drive/data/label_np_2D.npy', y)

In [ ]:
# custom losss function - wighted categorical-crossentropy
## need testing
"""
A weighted version of categorical_crossentropy for keras (2.0.6). This lets you apply a weight to unbalanced classes.
@url: https://gist.github.com/wassname/ce364fddfc8a025bfab4348cf5de852d
@author: wassname
"""
from keras import backend as K
def weighted_categorical_crossentropy(weights):
    """
    A weighted version of keras.objectives.categorical_crossentropy
    
    Variables:
        weights: numpy array of shape (C,) where C is the number of classes
    
    Usage:
        weights = np.array([0.5,2,10]) # Class one at 0.5, class 2 twice the normal weights, class 3 10x.
        loss = weighted_categorical_crossentropy(weights)
        model.compile(loss=loss,optimizer='adam')
    """
    
    weights = K.variable(weights)
        
    def loss(y_true, y_pred):
        # scale predictions so that the class probas of each sample sum to 1
        y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
        # clip to prevent NaN's and Inf's
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        # calc
        loss = y_true * K.log(y_pred) * weights
        loss = -K.sum(loss, -1)
        return loss
    
    return loss

Using TensorFlow backend.


In [ ]:
# model definition
def define_model(input_shape, num_of_class):
  regularizer = regularizers.l1_l2(l1=0.000, l2=0.000)
  x = input_shape[0]
  y = input_shape[1]

  model = Sequential()
  model.add(Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1), data_format='channels_first', input_shape=(1, x, y), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2), data_format='channels_first'))
  model.add(Dropout(rate = 0.1))

  model.add(Conv2D(filters=128, kernel_size=(3, 3), strides=(1, 1), activation='relu', data_format='channels_first'))
  model.add(MaxPooling2D(pool_size=(2, 2), data_format='channels_first'))
  model.add(Dropout(rate = 0.1))

  model.add(Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), activation='relu', data_format='channels_first'))
  model.add(MaxPooling2D(pool_size=(2, 2), data_format='channels_first'))
  model.add(Dropout(rate = 0.1))
            
  model.add(Flatten())

  model.add(Dense(units =1024, activation = 'relu', kernel_regularizer = regularizer))
  model.add(Dense(units = 1024, activation = 'relu', kernel_regularizer = regularizer))
  model.add(Dense(units = 512, activation = 'relu', kernel_regularizer = regularizer))
            
  model.add(Dense(units = num_of_class, activation = 'softmax'))

  print(model.summary())

  return model

In [ ]:
# generate data and label
def generate_dataset(data, label, shape):
 
  data = preprocess_data(data)
  data, label = shuffle_data(data, label)
  data = data_pd_to_np(data, shape)
  label = label_pd_to_np(label)

  return data, label

In [ ]:
# let's generate data and train model
classes = ['GBM', 'OV', 'LUAD', 'LUSC', 'PRAD', 'UCEC', 'BLCA', 'TGCT', 'ESCA', 'PAAD', 'KIRP', 'LIHC', 'CESC', 'SARC', 'BRCA', 'THYM', 'MESO', 'COAD', 'STAD', 'SKCM', 'CHOL', 'KIRC', 'THCA', 'HNSC', 'LAML', 'READ', 'LGG', 'DLBC', 'KICH', 'UCS', 'ACC', 'PCPG', 'UVM']
shape = (150, 150)

x, y = generate_dataset(data, label, shape)

model = define_model(shape, y.shape[1])

In [ ]:
# define learning process

sgd = optimizers.SGD(lr=0.0001, momentum=0.0, decay=0.0, nesterov=True)
#adam  = optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

# weights = [174, 309, 576, 554, 550, 567, 427, 156, 196, 183, 323, 424, 310, 265, 1218, 122, 87, 495, 450, 474, 45, 606, 572, 566, 173, 171, 534, 48, 91, 57, 79, 187, 80]
# weights = [ weight/11069 for weight in weights]

# model.compile(loss = weighted_categorical_crossentropy(weights), optimizer = sgd, metrics=['accuracy'])
model.compile(loss = 'categorical_crossentropy', optimizer = sgd, metrics=['accuracy'])

In [ ]:
# train model

history = model.fit(x, y, validation_split = 0.1, batch_size=32, epochs=80, verbose=1, shuffle=True)
with open('/content/drive/My Drive/results/check', 'wb') as file_pi:
  pickle.dump(history.history, file_pi)

Epoch 1/80
312/312 [==============================] - 12s 38ms/step - loss: 3.8339 - accuracy: 0.0827 - val_loss: 3.0859 - val_accuracy: 0.1852
Epoch 2/80
312/312 [==============================] - 11s 36ms/step - loss: 3.1657 - accuracy: 0.1406 - val_loss: 2.7950 - val_accuracy: 0.4110
Epoch 3/80
312/312 [==============================] - 11s 36ms/step - loss: 2.9395 - accuracy: 0.2118 - val_loss: 2.4706 - val_accuracy: 0.4670
Epoch 4/80
312/312 [==============================] - 11s 36ms/step - loss: 2.6654 - accuracy: 0.2929 - val_loss: 2.0711 - val_accuracy: 0.4851
Epoch 5/80
312/312 [==============================] - 11s 36ms/step - loss: 2.3512 - accuracy: 0.3958 - val_loss: 1.6401 - val_accuracy: 0.5772
Epoch 6/80
312/312 [==============================] - 11s 36ms/step - loss: 1.9970 - accuracy: 0.4956 - val_loss: 1.2557 - val_accuracy: 0.7173
Epoch 7/80
312/312 [==============================] - 11s 36ms/step - loss: 1.6790 - accuracy: 0.5756 - val_loss: 1.0020 - val_accuracy:

In [ ]:
# load models and weights

model_json = model.to_json()
model.save("/content/drive/My Drive/results/model_without_featreduction_2D_1.h5")